In [223]:
import sqlite3
import pandas as pd
from mpl_toolkits.mplot3d import axes3d
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
from datetime import datetime
from datetime import datetime, timedelta

con = sqlite3.connect('C:\ApsimX\Prototypes\MultiPoreWater/ReportDetail.db')
APSIMData = pd.read_sql("Select * from Report,Simulations where Report.SimulationID = Simulations.ID",con, index_col = ['TimeStep'])
APSIMData = APSIMData.drop(labels = ['SimulationID', 'ZoneName','ID'],axis=1)
PoreWater = APSIMData.ix[:,[X for X in APSIMData.columns if 'PW' in X]]
ProfileData = pd.read_sql("Select * from Initial,Simulations where Initial.SimulationID = Simulations.ID",con)
Thickness = ProfileData.ix[:,[X for X in ProfileData.columns if 'Thick' in X]]
Timing = APSIMData.ix[:,0:3]
Timing.ix[:,'DateTime'] = [datetime.strptime(Timing.ix[X,0], '%Y-%m-%d %H:%M:%S') for X in Timing.index]
Timing.ix[:,'Date0'] = [Timing.ix[X,'DateTime'] - timedelta(hours=12) for X in Timing.index]
Timing.ix[:,'DateTime'] = [Timing.ix[X,'Date0'] + timedelta(hours=int(Timing.ix[X,'Hour'])) for X in Timing.index]
Timing.ix[:,'Now'] = [datetime.strftime(Timing.ix[X,'DateTime'],'%Y-%m-%d %H:%M:%S') for X in Timing.index]

Depth = []
depth = 0
for l in Thickness.columns:
    Depth.append(depth)
    depth += Thickness.ix[:,l][0]
LayerNum = 7
PoreComNum = 7

LayerIndices = []
for i in range(1,LayerNum +1):
    LayerIndices.append('L'+str(i))
    
CompartmentIndices = []
for i in range(1,PoreComNum +1):
    CompartmentIndices.append('C'+str(i))    

PoreColumnIndex = pd.MultiIndex.from_product([LayerIndices,CompartmentIndices],names=['Layer','PoreComponent'])
PoreWater.columns = PoreColumnIndex

def WetnessData(i):
    Wetness = []
    for l in range(1,LayerNum+1):
        Wetness.append(list(PoreWater.ix[i+1,'L'+str(l)].values))
    return np.array(Wetness)

In [222]:
fig = plt.figure()
ax = axes3d.Axes3D(fig)

pores = range(1,PoreComNum+1)
layers = Depth
Pores, Layers = np.meshgrid(pores, layers)
Wetness = WetnessData(1)
wframe = ax.plot_wireframe(Layers,Pores, Wetness)
ax.set_zlim(0,1)

def update(i, ax, fig):
    ax.cla()
    Wetness = WetnessData(i)
    wframe = ax.plot_wireframe(Layers, Pores, Wetness)
    ax.set_zlim(0,1)
    ax.set_xlabel('Profile Depth (mm)')
    ax.set_ylabel('L pores --> S pores')
    ax.set_zlabel('Relative Water Content')
    return wframe,

ani = animation.FuncAnimation(fig, update, 
       frames=xrange(PoreWater.index.size), 
       fargs=(ax, fig), interval=0.001)
plt.show()

In [224]:
fig = plt.figure()
ax = fig.gca(projection='3d')

pores = range(1,PoreComNum+1)
layers = Depth
Pores, Layers = np.meshgrid(pores, layers)
Wetness = WetnessData(1)
surface = ax.plot_surface(Layers, Pores, Wetness, cstride=1, rstride=1, cmap=cm.jet)
ax.set_zlim3d(0.0, 1.0)

def update(i, ax, fig):
    ax.cla()
    Wetness = WetnessData(i)
    surface = ax.plot_surface(Layers, Pores, Wetness, cstride=1, rstride=1, cmap=cm.jet)
    ax.set_zlim(0,1)
    ax.set_xlabel('Profile Depth (mm)')
    ax.set_ylabel('L pores --> S pores')
    ax.set_zlabel('Relative Water Content')
    ax.text(0,0,1.5,Timing.ix[i+1,'Now'])
    return surface,

ani = animation.FuncAnimation(fig, update, 
        frames=xrange(PoreWater.index.size), 
        fargs=(ax, fig), interval=0.001)
plt.show()